# Sagemaker Xor tutorial
---

## prerequisite
Tensorflow Estimator에 대해서 처음 접하신다면 아래 페이지를 먼저 참고하시는 것을 추천 드립니다.  
[Estimator xor tutorial link](http://blog.sagemaker.io/tensorflow/2018/02/02/xor.html)  

### directory 구조
---
directory내에 Tensorflow Estimator tutorial 과 다르게 추가된 파일은 xor_classifier.py  
입니다. 이 파일에 대해서는 아래에서 설명하도록 하겠습니다.

In [1]:
!tree

.
├── __init__.py
├── data
│   ├── xor_test.csv
│   └── xor_train.csv
├── xor-sagemaekr-train-evaluate.ipynb
├── xor_all.ipynb
└── xor_classifier.py

1 directory, 6 files


## Sagemaker에서 Tensorflow estimator를 사용할때의 차이점
---
1. input function을 작성할시 parameter로 s3 bucket의 위치를 전달받는다.
2. entry point 파일에 input function, estimator가 정의되어 있어야 한다.
3. input function에는 train, evaluation, serving을 위한 세가지 input function이 정의되 있어야 한다.
4. predefined estimator를 사용할 경우 estimator_fn을 작성하고 custom estimator를 사용할 경우 model_fn을 작성한다.
5. estimator config는 sagemaker에서 정의 하므로 run_config를 인자로 받는다. (**cluster spec**)

## S3에 Data Upload 하기
위 1번 항목에 서술한 것처럼 S3 bucket내에 학습, 평가에 사용할  
Data가 미리 upload 되어 있어야 합니다.  
Python Sagemaker SDK를 활용해서 csv 파일들을 Upload하겠습니다.
  

In [5]:
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

# path = local data의 위치
inputs = sagemaker_session.upload_data(path='data',
                                       bucket='tutorial-dudaji',
                                       key_prefix='xor/data')
print(inputs)

s3://tutorial-dudaji/xor/data


inputs 에 저장된 s3 location은 나중에 train, evaluate과정에서 전달됩니다.  

### Sagemaker Tensorflow Entry point 파일 해부
---
   
```python
import numpy as np
import os
import tensorflow as tf


# run_config를 인자로 받아 Sagemaker에서 정의한 configure에 따라서 학습
# default config와의 가장큰 차이는 cluster spec 차이
def estimator_fn(run_config, params):
    feature_columns = [
        tf.feature_column.numeric_column('x', shape=[2])]
    return tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                      hidden_units=[10, 20, 10],
                                      n_classes=2,
                                      config=run_config)


# Sagemaker에서 host할 때 필요한 input function 
def serving_input_fn(params):
    feature_spec = {
        'x': tf.FixedLenFeature(dtype=tf.float32, shape=[4])}
    return tf.estimator.export.build_parsing_serving_input_receiver_fn(
        feature_spec)()


#학습 할때 input function, training_dir에는 s3 bucket내 data의 위치가 전달된다.
def train_input_fn(training_dir, params):
    """Returns input function that would feed the model during training"""
    return _generate_input_fn(training_dir, 'xor_train.csv')


def eval_input_fn(training_dir, params):
    """Returns input function that would feed the model during evaluation"""
    return _generate_input_fn(training_dir, 'xor_test.csv')


def _generate_input_fn(training_dir, training_filename):
    data_file = os.path.join(training_dir, training_filename)
    train_set = np.loadtxt(fname=data_file, delimiter=',')
    
    return tf.estimator.inputs.numpy_input_fn(
        x={"x": train_set[:, 0:-1]},
        y=np.array(train_set[:, [-1]]),
        num_epochs=None,
        shuffle=True)()
```

### Sagemaker로 Estimator Train, Evaluate 시키기
---

In [11]:
from sagemaker.tensorflow import TensorFlow
from sagemaker import get_execution_role

role = get_execution_role()

xor_classifier = TensorFlow(entry_point='xor_classifier.py',
                            role=role,
                            train_instance_count=1,
                            train_instance_type='ml.c4.xlarge',
                            training_steps=1000,
                            evaluation_steps=100)

### 학습 시작
---
학습시 fit을 실행시키면 되며 이때 인자로 data가 있는 s3 의 위치를 넘겨줍니다.
이 위치가 input_fn에 전달됩니다.

![image.png](attachment:image.png)

In [12]:
xor_classifier.fit(inputs, run_tensorboard_locally=True)

INFO:sagemaker:TensorBoard 0.1.7 at http://localhost:6006
INFO:sagemaker:Created S3 bucket: sagemaker-us-east-1-728064587231
INFO:sagemaker:Creating training-job with name: sagemaker-tensorflow-py2-cpu-2018-01-24-08-29-48-918


.........................................................
executing startup script (first run)
2018-01-24 08:35:23,957 INFO - root - running container entrypoint
2018-01-24 08:35:23,957 INFO - root - starting train task
2018-01-24 08:35:25,558 INFO - botocore.vendored.requests.packages.urllib3.connectionpool - Starting new HTTP connection (1): 169.254.170.2
2018-01-24 08:35:26,444 INFO - botocore.vendored.requests.packages.urllib3.connectionpool - Starting new HTTPS connection (1): s3.amazonaws.com
2018-01-24 08:35:26,615 INFO - botocore.vendored.requests.packages.urllib3.connectionpool - Starting new HTTPS connection (1): s3.amazonaws.com
INFO:tensorflow:----------------------TF_CONFIG--------------------------
INFO:tensorflow:{"environment": "cloud", "cluster": {"master": ["algo-1:2222"]}, "task": {"index": 0, "type": "master"}}
INFO:tensorflow:---------------------------------------------------------
INFO:tensorflow:going to training
2018-01-24 08:35:26,680 INFO - root - creating Ru

INFO:tensorflow:Validation (step 1000): loss = 86.3331, accuracy_baseline = 0.500391, global_step = 1, auc = 1.0, prediction/mean = 0.522813, label/mean = 0.499609, average_loss = 0.674477, auc_precision_recall = 1.0, accuracy = 0.499609
INFO:tensorflow:Saving checkpoints for 1000 into s3://sagemaker-us-east-1-728064587231/sagemaker-tensorflow-py2-cpu-2018-01-24-08-29-48-918/checkpoints/model.ckpt.
INFO:tensorflow:Loss for final step: 0.0106414.
INFO:tensorflow:Starting evaluation at 2018-01-24-08:35:39
INFO:tensorflow:Restoring parameters from s3://sagemaker-us-east-1-728064587231/sagemaker-tensorflow-py2-cpu-2018-01-24-08-29-48-918/checkpoints/model.ckpt-1000
INFO:tensorflow:Evaluation [1/100]
INFO:tensorflow:Evaluation [2/100]
INFO:tensorflow:Evaluation [3/100]
INFO:tensorflow:Evaluation [4/100]
INFO:tensorflow:Evaluation [5/100]
INFO:tensorflow:Evaluation [6/100]
INFO:tensorflow:Evaluation [7/100]
INFO:tensorflow:Evaluation [8/100]
INFO:tensorflow:Evaluation [9/100]
INFO:tensorflow